### Notebook Responsável por buscar as partidas do campeonato do ano corrente

In [21]:
import requests
import json
import datetime
import pandas as pd

In [22]:
%run ./database.ipynb

In [23]:
def request_last_turn_from_cartola_api():
    home_page = requests.get('https://api.cartolafc.globo.com/partidas')
    home_json = json.loads(home_page.content)
    last_turn = home_json['rodada']
    return last_turn

In [24]:
def create_matches_dataframe(last_turn):
    year = datetime.date.today().year
    matches = []
    
    for turn in range(1, last_turn):
        turn_page = requests.get(f'https://api.cartolafc.globo.com/partidas/{turn}')
        turn_json = json.loads(turn_page.content)
        turn_matches = turn_json['partidas']
        for match_data in turn_matches:
            match = {
                'match_id': match_data['partida_id'],
                'turn': turn,
                'home_id': match_data['clube_casa_id'],
                'visitor_id': match_data['clube_visitante_id'],
                'date': match_data['partida_data'],
                'timestamp': match_data['timestamp'],
                'local': match_data['local'],
                'valid': match_data['valida'],
                'home_goal': match_data['placar_oficial_mandante'],
                'visitor_goal': match_data['placar_oficial_visitante'],
                'year': year
            }
            matches.append(match)

    return pd.DataFrame(matches)

In [25]:
def collect_matches_and_save_to_database(engine):
    last_turn = request_last_turn_from_cartola_api()
    df_matches = create_matches_dataframe(last_turn)
    save_data(engine, 'match', df_matches)
    return count_data(engine, 'match').fetchall()